# Calibration: yield-curve fitting

In [20]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds
from copy import copy
from scipy.optimize import OptimizeResult, minimize
from optimparallel import minimize_parallel
from monte_carlo import MonteCarlo
from ecir import JumpCIR

In [21]:
class Calibration:
    def __init__(self, model_class, n: int, m: int, r0: float, model_params: dict, optimize_args: tuple, calibrate_exact=False, seed=93756826):
        """
        model_class: The class of the interest rate model.
        n and m: Parameters for the number of paths in Monte Carlo simulations.
        r0: The initial interest rate.
        model_params: Initial parameters of the model.
        optimize_args: A tuple of the names of parameters to optimize.
        calibrate_exact: Whether to use exact calculations instead of Monte Carlo simulations.
        seed: Used to set the random seed to ensure reproducibility of results.
        """
        self.n = n
        self.m = m
        self.r0 = r0
        self.seed = seed
        self.model_class = model_class
        self.model_params = model_params
        self.optimize_args = optimize_args
        self.calibrate_exact = calibrate_exact

    def _calculate_error(self, optimize_params: np.array, Ts: np.array, prices: np.array) -> float:
        """
        The error between the model prices and market prices given parameters.
        For each given maturity and price, calculates the difference between model prices and market prices, generating simulation paths with Monte Carlo simulations (unless exact calculation is chosen).
        Then calculates the Euclidean norm of the price errors for all maturities on each date, and returns the average of all date errors.
        The first value of optimize_params is always r0.
        """
        model_params = copy(self.model_params)
        for arg, param in zip(self.optimize_args, optimize_params):  # Pairing parameter names and values together
            model_params[arg] = param  # Assigning optimization parameter values to corresponding parameter names, updating values in the model parameter copy
        model = self.model_class(model_params)  # Forming a new interest rate model with updated parameters
        mc = MonteCarlo(model)
        dates, maturities = prices.shape
        date_errors = np.empty(dates)  # Errors for each date
        maturity_errors = np.empty(maturities)  # Errors for each maturity
        j = 0  # Current date index
        
        for date in range(dates):
            i = 0
            for price, T in zip(prices[date, :], Ts):
                np.random.seed(self.seed)
                # Set the random seed to ensure that the paths generated in the mc simulation are reproducible, thus reducing random variation in results.
                
                if self.calibrate_exact:  # Exact calculation of error: model price versus actual price
                    maturity_errors[i] = model.exact(r0=self.r0, T=T) - price
                else:
                    maturity_errors[i] = mc._simulate_paths_anti(m=self.m, r0=self.r0, n=self.n, T=T)[0] - price
                i += 1
            date_errors[j] = np.linalg.norm(maturity_errors)  # Length of the error vector
            j += 1
        return np.mean(date_errors)  # Overall magnitude of the error between the model and the market across all maturities

    def calibrate(self, initial_values: tuple, Ts: np.array, prices: np.array, bounds: Bounds) -> OptimizeResult:
        """
        initial_values, np.array: same length as optimize_args,
        Ts, np.array: array of maturities to fit,
        prices, np.array: array of prices to fit (same number of columns as Ts),
        bounds, scipy.optimize.Bounds or None: linear bounds on the solution to ensure the algorithm doesn't waste time searching for unreasonable parameter values (e.g., negative values).
        """
        error_function = lambda optimize_params: self._calculate_error(optimize_params, Ts=Ts, prices=prices)
        optimal = minimize(error_function, initial_values, method="L-BFGS-B", bounds=bounds)
        return optimal


In [26]:
# Load CSV and set DATE as index
df = pd.read_csv("Data Folder/DGS_30.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df.index.name = 'DATE'

# Select yields for a specific date, '2019-04-17'
selected_date = '2019-04-17'
yields = df.loc[selected_date].astype(float) / 100

# Create a DataFrame for the selected yields
yield_data = pd.DataFrame({
    'Yield': yields.values,
    'Maturity': np.arange(1, len(yields) + 1)
})

# Calculate cumulative yield and bond price
yield_data["Cum. Yield"] = yield_data["Yield"] * yield_data["Maturity"]
yield_data["Price"] = np.exp(-yield_data["Cum. Yield"])

# Get prices and maturities for pricing the bond
prices = yield_data["Price"].values
Ts = yield_data["Maturity"].values

prices = prices.reshape(-1, 1)
dates, maturities = prices.shape

In [27]:
# Initialize model parameters
initial_model_params = {
    "kappa": 0.5,
    "mu_r": 0.03,
    "sigma": 0.03,
    "mu": 0,
    "gamma": 0.01,
    "h": 10,
}

In [28]:
optimize_args = ("kappa", "mu_r", "sigma", "gamma")
bounds = Bounds([0.001, 0.001, 0.01, 0.01], [1, 0.2, 0.2, 0.2])

In [29]:
if __name__ == "__main__":
    calibrator = Calibration(
        model_class=JumpCIR, 
        n=100, m=26, r0=0.001,
        model_params=initial_model_params, optimize_args=optimize_args)
    optimal_params = calibrator.calibrate(
        initial_values=(initial_model_params["kappa"], initial_model_params["mu_r"], 
                        initial_model_params["sigma"], initial_model_params["gamma"]),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )
    print(optimal_params)

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.2609950323953853
        x: [ 1.000e+00  2.000e-01  1.000e-02  1.000e-02]
      nit: 5
      jac: [-4.103e-02 -2.851e-01  5.401e-05  1.582e-03]
     nfev: 45
     njev: 9
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
